In [6]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [7]:
# Fonction pour lire et prétraiter les vidéos
def read_and_process_videos(directory, label, image_size=(224, 224)):
    frames = []
    for filename in os.listdir(directory):
        if filename.endswith(".mp4"):
            video_path = os.path.join(directory, filename)
            cap = cv2.VideoCapture(video_path)
            while(cap.isOpened()):
                ret, frame = cap.read()
                if ret:
                    frame = cv2.resize(frame, image_size)
                    frames.append(frame)
                else:
                    break
            cap.release()
    return frames, [label] * len(frames)

In [8]:
# Chemin vers le répertoire contenant les vidéos de la classe 0
class_0_directory = "E:/AI DB/LSPD/Videos/normal_final"
class_0_frames, class_0_labels = read_and_process_videos(class_0_directory, label=0)

# Chemin vers le répertoire contenant les vidéos de la classe 1
class_1_directory = "E:/AI DB/LSPD/Videos/porn_final"
class_1_frames, class_1_labels = read_and_process_videos(class_1_directory, label=1)

# Concaténer les données de deux classes
frames = np.array(class_0_frames + class_1_frames)
labels = np.array(class_0_labels + class_1_labels)

# Diviser les données en ensembles d'entraînement et de test
train_frames, test_frames, train_labels, test_labels = train_test_split(frames, labels, test_size=0.2, random_state=42)

In [14]:
# Construction du modèle CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

In [15]:
# Compilation du modèle
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [16]:
# Entraînement du modèle
model.fit(train_frames, train_labels, epochs=6, batch_size=20, validation_split=0.1)

Epoch 1/6
8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.5183 - loss: 535.3773 - val_accuracy: 0.3750 - val_loss: 372.0639
Epoch 2/6
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 499ms/step - accuracy: 0.6881 - loss: 142.1169 - val_accuracy: 0.3750 - val_loss: 296.0150
Epoch 3/6
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 461ms/step - accuracy: 0.4674 - loss: 129.4973 - val_accuracy: 0.8750 - val_loss: 0.8087
Epoch 4/6
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 492ms/step - accuracy: 0.8739 - loss: 2.3644 - val_accuracy: 0.9375 - val_loss: 0.3711
Epoch 5/6
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 445ms/step - accuracy: 0.8766 - loss: 2.2797 - val_accuracy: 0.8125 - val_loss: 1.9118
Epoch 6/6
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 434ms/step - accuracy: 0.8408 - loss: 1.2106 - val_accuracy: 0.8750 - val_loss: 0.4219


In [17]:
# Évaluation du modèle
test_loss, test_acc = model.evaluate(test_frames, test_labels)
print('Test accuracy:', test_acc)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.8479 - loss: 0.7727
Test accuracy: 0.8500000238418579
